In [ ]:
'''
Dual Non-Local Means: a two-stage information-theoretic filter for image denoising
'''

import warnings
import time
import skimage
import skimage.io
import numpy as np
from skimage.filters.edges import convolve
from skimage.transform import resize
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import adapted_rand_error
from skimage.metrics import variation_of_information
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral, denoise_wavelet, estimate_sigma)

# Para evitar warning de divisão por zero
warnings.simplefilter(action='ignore')


'''
Parâmetros:

	P1m, P1v: patches centrais de referência das médias e variâncias
	P2m, P2v: patches das médias e variâncias pertencente a janela de busca
'''
def divergenciaKL(P1m, P1v, P2m, P2v):

	# Versão mais rápida de calcular
	dKL = np.sum( (1/(4*P1v*P2v))*( (P1v - P2v)**2 + ((P1m - P2m)**2)*(P1v + P2v) ) )

	return dKL

'''
Parâmetros:

	P1m, P1v: patches centrais de referência das médias e variâncias
	P2m, P2v: patches das médias e variâncias pertencente a janela de busca
'''
def Bhattacharyya(P1m, P1v, P2m, P2v):

	# Calcula o coeficiente de Bhattacharyya (np.sum é aqui ou embaixo?)
	cBhat = np.sqrt( (2*np.sqrt(P1v)*np.sqrt(P2v)/(P1v + P2v) )*np.exp( -0.25* ( ((P1m - P2m)**2)/(P1v + P2v) ) ) )
	# Calcula distância de Bhattacharyya
	dBhat = np.sum( -np.log(cBhat) )

	return dBhat

'''
Parâmetros:

	P1m, P1v: patches centrais de referência das médias e variâncias
	P2m, P2v: patches das médias e variâncias pertencente a janela de busca
'''
def Hellinger(P1m, P1v, P2m, P2v):

	# Calcula o coeficiente de Bhattacharyya (np.sum é aqui ou embaixo?)
	cBhat = np.sqrt( (2*np.sqrt(P1v)*np.sqrt(P2v)/(P1v + P2v) )*np.exp( -0.25* ( ((P1m - P2m)**2)/(P1v + P2v) ) ) )
	# Calcula distância de Hellinger
	dHell = np.sum( 1 - cBhat )

	return dHell

'''
Parâmetros:

	img: imagem ruidosa de entrada
	h: parâmetro que controla o grau de suavização (quanto maior, mais suaviza)
	f: tamanho do patch (2f + 1 x 2f + 1) -> se f = 3, então patch é 7 x 7
	t: tamanho da janela de busca (2t + 1 x 2t + 1) -> se t = 10, então janela de busca é 21 x 21

'''
def NLM_KL(img, h, f, t):

	# Dimenssões espaciais da imagem
	m, n = img.shape

	# Cria imagem de saída
	filtrada = np.zeros((m, n))

	# Cria matrizes para armazenar médias e variâncias
	matriz_m = np.zeros((m, n))
	matriz_v = np.zeros((m, n))

	# Estima médias de maneira não-local
	# O parâmetro h dessa pré filtragem é diferente do h usado no filtro proposto! Em geral, bem maior!

	# Estima médias de maneira não local com NLM padrão
	matriz_m = NLM(img, 70, 2, 3)		# default = 70, testei 60 em alguns [50, 55, 60, 65, 70]

	# Problema de valor de contorno: replicar bordas
	img_n = np.pad(img, ((f, f), (f, f)), 'symmetric')

	# Estima variâncias locais
	for i in range(m):
		for j in range(n):
			im = i + f
			jn = j + f
			matriz_v[i, j] = img_n[im-f:im+f+1, jn-f:jn+f+1].var()

	# Replica bordas
	matriz_m = np.pad(matriz_m, ((f, f), (f, f)), 'symmetric')
	matriz_v = np.pad(matriz_v, ((f, f), (f, f)), 'symmetric')

	# Loop principal do NLM
	for i in range(m):
		for j in range(n):

			im = i + f;   # compensar a borda adicionada artificialmente
			jn = j + f;   # compensar a borda adicionada artificialmente

        	# Obtém o patch ao redor do pixel corrente em matriz_m e matriz_v
			W1m = matriz_m[im-f:(im+f)+1, jn-f:(jn+f)+1]
			W1v = matriz_v[im-f:(im+f)+1, jn-f:(jn+f)+1]

        	# Calcula as bordas da janela de busca para o pixel corrente
			rmin = max(im-t, f);  # linha inicial
			rmax = min(im+t, m+f);  # linha final
			smin = max(jn-t, f);  # coluna inicial
			smax = min(jn+t, n+f);  # coluna final

        	# Calcula média ponderada
			NL = 0      # valor do pixel corrente filtrado
			Z = 0       # constante normalizadora

        	# Loop para todos os pixels da janela de busca
			for r in range(rmin, rmax):
				for s in range(smin, smax):

                	# Obtém o patch ao redor do pixel a ser comparado em matriz_m e matriz_v
					W2m = matriz_m[r-f:(r+f)+1, s-f:(s+f)+1]
					W2v = matriz_v[r-f:(r+f)+1, s-f:(s+f)+1]

                	# Calcula a divergência KL simetrizada
					dKL = divergenciaKL(W1m, W1v, W2m, W2v)
					#dKL = Bhattacharyya(W1m, W1v, W2m, W2v)
					#dKL = Hellinger(W1m, W1v, W2m, W2v)

                	# Calcula a medida de similaridade
					sij = np.exp(-dKL/(h**2))

                	# Atualiza Z e NL
					Z = Z + sij
					NL = NL + sij*img_n[r, s]

        	# Normalização do pixel filtrado
			filtrada[i, j] = NL/Z

	return filtrada

In [ ]:
#%%%%%%%%%%% Início do script

# Imagens em tons de cinzas
name = 'lena.bmp'

# Lista de imagens
lista = [name]

# Aplica NLM-KL
for nome in lista:

	print('********** Imagem: %s' %nome)

	img = skimage.io.imread(nome)

	# Verifica se imagem é colorida (tem mais de um canal)
	if len(img.shape) > 2:
		img = skimage.color.rgb2gray(img)
		img = 255*img
		#img = img.astype(np.uint8)

	# Se imagem original não for uint8
	if img.max() > 255:
		img = img/img.max()
		img = 255*img
		#img = img.astype(np.uint8)

	m, n = img.shape

	# Setar para True para redimensionar as imagens para 128 x 128 (diminui custo computacional)
	reshape = False

	if reshape:
		# Converte todas as imagens para 128 x 128
		if (m == 256) and (n == 256):
			img = resize(img, (m//2, n//2), anti_aliasing=False, preserve_range=True)
			img = img.astype(np.uint8)    # Converte para uint8    
		elif (m == 512) and (n == 512):
			img = resize(img, (m//4, n//4), anti_aliasing=False, preserve_range=True)
			img = img.astype(np.uint8)    # Converte para uint8
		elif (m == 1024) and (n == 1024):
			img = resize(img, (m//8, n//8), anti_aliasing=False, preserve_range=True)
			img = img.astype(np.uint8)    # Converte para uint8		
	else:
		img = img.astype(np.uint8)

	m, n = img.shape
	# Desvio padrão do ruído
	

	# Parêmtros do NLM
	f = 4	
	t = 7	

	#%%%%%%%% Non-Local Means
	# O parâmetro h do filtro NLM_KL é diferente do utilizado na pré filtragem para obtenção das médias não locais
	# Em geral, é bem menor. (de 1.0 a 2.0. Controla a suavização (global)
	if f == 1:
		lista_h = [0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6] 	# 256 x 256, H = 50
	elif f == 2:
		lista_h = [1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0] 	# 256 x 256, H = 60, 70  (*)
	elif f == 3:
		lista_h = [2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8]		# 256 x 256, H = 65, 70 
	elif f == 4:
		lista_h = [2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3] 			
	elif f == 5:
		lista_h = [3.6, 3.8, 4.0, 4.2, 4.4, 4.6] 					# 256 x 256
		
	print('######### NLM-KL #################')
	
	inicio = time.time()
	filtrada = NLM_KL(ruidosa, h=3.0, f=4, t=7)
	fim = time.time()

	print('Tempo de execução: %.2f segundos' %(fim - inicio))
	print('h = %.2f' %h)

	# Salva imagem no arquivo de saída
	# Converte para uint8 (8 bits)
	output = filtrada.astype(np.uint8)
	file = nome[14:]
	limite = file.find('.')
	filename = file[:limite] + '_NLM_KL_OK_' + str(h) + '.png'
	skimage.io.imsave(filename, output)

	# Calcula PSNR
	p = PSNR(img, output)
	print('PSNR: %f' %p)

	# Calcula SSIM
	s = SSIM(img, output)
	print('SSIM: %f' %s)

	print()

In [14]:
import sys
from pathlib import Path
PROJECT_ROOT = Path(__file__).resolve().parents[1]
sys.path.append(str(PROJECT_ROOT))

from Gaussian_DUALNLM import generate_gaussian_experiment_low_dual_nlm

from functions.Utils import ensure_output_dirs

if __name__ == '__main__':

    # Base output directory for the low-noise experiment results
    root_dir_output_low = Path('/workspace/data/output/set12/low_noisy') 

    # Directory containing the input images used in the experiment
    dir_images_general = Path('/workspace/data/input/set12')

    # Ensure required output directories exist
    ensure_output_dirs(root_dir_output_low)

    # Dictionary of parameters passed to the experiment generator
    parameters = {

        # Paths for reading input and saving results
        'root_dir_output_low': str(root_dir_output_low),
        'dir_images_general': str(dir_images_general),

        # Output folders for each filtering method

        'dir_out_dualnlm': str(root_dir_output_low / 'DUALNLM'),
        'dir_out_results': str(root_dir_output_low / 'results'),

        # Filenames for serialized results (pickle/XLSX)
        'name_pickle_dual_nlm_output_low': 'array_dual_nlm_low_filtereds.pkl',    
        'name_results_xlsx_dual_nlm_output_low':'dual_nlm_low_filtereds.xlsx',
        'pickle_results_summary_low': '/workspace/data/output/set12/low_noisy/results/array_nln_low_filtereds.pkl',

        # Algorithmic parameters used internally by the experiment
        'f': 4,        # Patch radius
        't': 7,        # Search window radius    
    }

    # Execute the low-noise Gaussian experiment
    generate_gaussian_experiment_low_dual_nlm(parameters)


NameError: name '__file__' is not defined